In [1]:
import numpy as np
import patsy as pt
import pandas as pd
from sklearn import model_selection, metrics, linear_model

### Algorithm

- In the forward direction: refit the model with each of the remaining predictors and incorporate the one which brings about the greatest improvement in performance.
- In the reverse direction: refit the model dropping each of the remaining predictors and remove the one which leads to the smallest reduction in performance.
- There are $1 + \frac{p(1 + p)}{2}$ combinations to consider.

In [125]:
def stepwise(X, y, forward=True):
    p = X.shape[1]
    print('Searching over {} models'.format(int(1 + ((p * (1 + p))/2))))
    # begin with no selections, `best` will be mutated as the algorithm runs.
    # `best` is the vector representing the current best combination of predictors.
    best = np.repeat(0, p)
    
    selection = best if forward else np.logical_not(best)
    Xf = X @ np.diag(selection)
    model = linear_model.LinearRegression().fit(Xf, y)
    score = model.score(X, y)

    scores = [score]
    models = [model]
    vectors = [best]
    
    while not np.array_equal(best, np.repeat(1, p)):
        selection_matrix = np.identity(p) @ np.diag(np.logical_not(best))
        best_vector = None
        best_score = None
        best_model = None
        for c in selection_matrix.T:
            if np.array_equal(c, np.zeros(len(c))):
                continue
            trace = c + best
            selection = trace if forward else np.logical_not(trace)
            Xf = X @ np.diag(selection)
            model = linear_model.LinearRegression().fit(Xf, y)
            score = model.score(X, y)
            if best_score == None or score > best_score:
                best_vector = c
                best_score = score
                best_model = model
        best = best + best_vector
        scores.append(best_score)
        models.append(best_model)
        vectors.append(best)
    return scores, models, vectors

n = 100
p = 5
X = np.random.normal(1,1,(n, p))
eps = np.random.normal(1,1, n).reshape(-1,1)
beta = np.arange(1, p + 1).reshape(-1,1)
y = (X @ beta) + eps

scores, models, vectors = stepwise(X,y, forward=False)

scores

# TODO
# - remove the nasty if at the top of the loop
# - add comments
# - factor out repeated model fit / selection code

Searching over 16 models


[0.9851503701807045,
 0.9674423640652615,
 0.9027743210969267,
 0.7958948240026296,
 0.599751372208384,
 0.0]

### Generate some data

Example 1: All predictors are equally associated with the outcome

In [123]:
n = 1000
p = 10
beta = np.repeat(2, p).reshape(-1, 1)
X = np.random.normal(0, 1, (n, p))
y = X @ beta
#train, test = model_selection.train_test_split(X, test_size=0.2)

#stepwise(train)